In [1]:
!pip install openpyxl

In [2]:
#dependencies
import pandas as pd
import datetime
from sqlalchemy import create_engine
import glob

In [3]:
# U.S. Gasoline and Diesel Retail Prices 1995-2021
csv_file = "PET_PRI_GND_DCUS_NUS_W.csv"
us_gas_price_df = pd.read_csv(csv_file)
us_gas_price_df.head(5)

,Date,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


In [4]:
#renaming the columns
gas_prices_df = us_gas_price_df.rename(columns={"A1":"All Grades All Formulations",
                                                     "A2":"All Grades Conventional",
                                                     "A3":"All Grades Reformulated",
                                                     "R1":"Regular All Formulations",
                                                     "R2":"Regular Conventional", 
                                                     "R3":"Regular Reformulated",
                                                     "M1":"Midgrade All Formulations",
                                                     "M2":"Midgrade Conventional", 
                                                     "M3":"Midgrade Reformulated", 
                                                     "P1":"Premium All Formulations", 
                                                     "P2":"Premium Conventional", 
                                                     "P3":"Premium Reformulated", 
                                                     "D1":"No 2 Diesel"}) 
gas_prices_df.head(5)

,Date,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


In [5]:
#pull year out of Date column in us_gas_price df
gas_prices_year = pd.DatetimeIndex(gas_prices_df['Date']).year
#add column back to dataframe in preparation for join
gas_prices_df['Year'] = gas_prices_year
gas_prices_df.head(5)

,Date,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel,Year
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104,1995
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102,1995
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100,1995
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095,1995
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090,1995


In [6]:
#average all the gas prices by the year via groupby
avg_annual_gas = gas_prices_df.groupby(['Year']).mean()

avg_annual_gas.head(5)

,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
Year,,,,,,,,,,,,,
1995,1.157712,1.143538,1.219615,1.110865,1.103308,1.162500,1.200635,1.196000,1.284904,1.297981,1.287154,1.368846,1.109154
1996,1.244528,1.230830,1.296849,1.199075,1.191925,1.241792,1.285170,1.280151,1.363962,1.380642,1.370679,1.438057,1.235283
1997,1.244250,1.227788,1.306808,1.199038,1.188692,1.252327,1.284192,1.276712,1.377269,1.380058,1.368481,1.446981,1.198308
1998,1.071712,1.056212,1.133135,1.029654,1.016654,1.077788,1.121904,1.105885,1.201058,1.213865,1.198788,1.275692,1.043885
1999,1.176058,1.155519,1.260154,1.135808,1.115981,1.195154,1.233596,1.207019,1.331115,1.319635,1.296558,1.399692,1.120519


In [7]:
#isolating the years of interest to match with the mpg dataset
avg_gas_price = avg_annual_gas.loc[2013:]

avg_gas_price.head(5)

,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
Year,,,,,,,,,,,,,
2013,3.575365,3.510519,3.707442,3.504865,3.442865,3.634981,3.662577,3.584038,3.814577,3.822462,3.765231,3.928692,3.921731
2014,3.436615,3.376423,3.559288,3.357673,3.298788,3.481365,3.539423,3.469154,3.675365,3.710192,3.663923,3.796250,3.824635
2015,2.519923,2.422692,2.718000,2.429212,2.333750,2.629288,2.644827,2.530865,2.865442,2.830192,2.750385,2.978538,2.706635
2016,2.249962,2.177173,2.398000,2.143250,2.070462,2.296096,2.394615,2.312904,2.552404,2.615750,2.567269,2.705750,2.303942
2017,2.527885,2.447769,2.691231,2.414615,2.333173,2.585500,2.681615,2.595096,2.848885,2.916500,2.864577,3.013019,2.650038


In [8]:
# file path
path = "complete_us_fuel_economy/"

all_files = glob.glob(path + "*.xlsx")

# empty list
files_list = []

# for loop to access the files
for filename in all_files:
    try:
        files_df =pd.read_excel(filename)
        files_list.append(files_df)
    
    except: PermissionError
        
# concate the files together
files_df = pd.concat(files_list, axis =0, ignore_index = True)

# isolating the columns of interest
files_clean = files_df.loc[:,["Model Year",
                     "Mfr Name",
                     "Division",
                     "City FE (Guide) - Conventional Fuel",
                     "Hwy FE (Guide) - Conventional Fuel",
                     "Comb FE (Guide) - Conventional Fuel",
                     "City Unadj FE - Conventional Fuel",
                     "Hwy Unadj FE - Conventional Fuel",
                     "Comb Unadj FE - Conventional Fuel",
                     "City Unrd Adj FE - Conventional Fuel",
                     "Hwy Unrd Adj FE - Conventional Fuel",
                     "Comb Unrd Adj FE - Conventional Fuel",
                     "Fuel Usage Desc - Conventional Fuel"]]
files_clean.head(5)

,Model Year,Mfr Name,Division,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unadj FE - Conventional Fuel,Hwy Unadj FE - Conventional Fuel,Comb Unadj FE - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Fuel Usage Desc - Conventional Fuel
0,NaN,aston martin,Aston Martin Lagonda Ltd,14,20,16,16.9502,28.0198,20.6508,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
1,NaN,aston martin,Aston Martin Lagonda Ltd,12,19,14,15.0065,25.6094,18.4349,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
2,NaN,Audi,Audi,12,19,15,15.3,26.8,18.9614,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
3,NaN,Audi,Audi,13,18,15,15.4,25.0451,18.6283,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)
4,NaN,Audi,Audi,13,20,16,15.8,24.8,18.8839,NaN,NaN,NaN,Gasoline (Premium Unleaded Recommended)


In [9]:
# dropping the NaN values and proform a groupby to average the annual mpg values
avg_mpg_df = files_clean.dropna().groupby("Model Year").mean()

avg_mpg_df.head(5)

,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel
Model Year,,,,,,
2013.0,19.574871,26.977625,22.246988,19.536401,26.912754,22.239994
2014.0,19.855351,27.418060,22.597826,19.839688,27.408190,22.599637
2015.0,20.153477,27.924061,22.961631,20.132416,27.876310,22.954063
2016.0,20.430338,28.164880,23.258862,20.424168,28.132987,23.241010
2017.0,20.569936,27.795016,23.245981,20.569113,27.772557,23.231466


In [10]:
#reset index and dropping the decimal number
index_avg_mpg = avg_mpg_df.reset_index()

avg_mpg_year= index_avg_mpg['Model Year'].astype(int)

# avg_mpg_year

# adding the year column back to the dataframe after dropping decimal and prepare for joining
index_avg_mpg['Year'] = avg_mpg_year

index_avg_mpg.head()

,Model Year,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel,Year
0,2013.0,19.574871,26.977625,22.246988,19.536401,26.912754,22.239994,2013
1,2014.0,19.855351,27.418060,22.597826,19.839688,27.408190,22.599637,2014
2,2015.0,20.153477,27.924061,22.961631,20.132416,27.876310,22.954063,2015
3,2016.0,20.430338,28.164880,23.258862,20.424168,28.132987,23.241010,2016
4,2017.0,20.569936,27.795016,23.245981,20.569113,27.772557,23.231466,2017


In [11]:
#dropping model year 
drop_model_yr = index_avg_mpg.drop(columns ='Model Year')

#dropping year 2022

average_mpg = drop_model_yr.drop(9)

#set index to year for joining

avg_mpg = average_mpg.set_index('Year')

avg_mpg.head(5)

,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel
Year,,,,,,
2013,19.574871,26.977625,22.246988,19.536401,26.912754,22.239994
2014,19.855351,27.418060,22.597826,19.839688,27.408190,22.599637
2015,20.153477,27.924061,22.961631,20.132416,27.876310,22.954063
2016,20.430338,28.164880,23.258862,20.424168,28.132987,23.241010
2017,20.569936,27.795016,23.245981,20.569113,27.772557,23.231466


In [12]:
#we can do a group by to get averages for all cars in dataframe
# avg_mpg = car_mpg.groupby(['Year']).mean()
avg_mpg

,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel
Year,,,,,,
2013,19.574871,26.977625,22.246988,19.536401,26.912754,22.239994
2014,19.855351,27.418060,22.597826,19.839688,27.408190,22.599637
2015,20.153477,27.924061,22.961631,20.132416,27.876310,22.954063
2016,20.430338,28.164880,23.258862,20.424168,28.132987,23.241010
2017,20.569936,27.795016,23.245981,20.569113,27.772557,23.231466
2018,20.651938,27.827132,23.311628,20.652907,27.809382,23.308930
2019,20.700787,27.645669,23.272441,20.681551,27.622681,23.266541
2020,20.675514,27.369007,23.175514,20.669748,27.370156,23.172568
2021,20.684943,27.033943,23.060052,20.693284,27.032448,23.065213


In [13]:
#for our convenience, the gas prices table of last 5 years are here so there's less scrolling. 
avg_gas_price

,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
Year,,,,,,,,,,,,,
2013,3.575365,3.510519,3.707442,3.504865,3.442865,3.634981,3.662577,3.584038,3.814577,3.822462,3.765231,3.928692,3.921731
2014,3.436615,3.376423,3.559288,3.357673,3.298788,3.481365,3.539423,3.469154,3.675365,3.710192,3.663923,3.796250,3.824635
2015,2.519923,2.422692,2.718000,2.429212,2.333750,2.629288,2.644827,2.530865,2.865442,2.830192,2.750385,2.978538,2.706635
2016,2.249962,2.177173,2.398000,2.143250,2.070462,2.296096,2.394615,2.312904,2.552404,2.615750,2.567269,2.705750,2.303942
2017,2.527885,2.447769,2.691231,2.414615,2.333173,2.585500,2.681615,2.595096,2.848885,2.916500,2.864577,3.013019,2.650038
2018,2.813189,2.714358,3.008151,2.718547,2.630528,2.904057,3.055434,2.934302,3.247057,3.294245,3.206377,3.412208,3.177642
2019,2.691327,2.566346,2.937596,2.603904,2.500519,2.826615,2.999288,2.825962,3.251269,3.251269,3.106346,3.420462,3.055827
2020,2.258250,2.144019,2.483962,2.167519,2.073712,2.369846,2.583000,2.428500,2.809231,2.834115,2.711365,2.978096,2.550808
2021,2.420250,2.309750,2.637500,2.334250,2.243750,2.526750,2.719000,2.562500,2.950750,2.975250,2.848750,3.125000,2.680500


In [14]:
#prepare connection info for postgres
#make sure creds match, esp database name
protocol = 'postgresql'
username = 'postgres'
password = 'admin'
host = 'localhost'
port = 5432
database_name = 'mydb'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [15]:
engine.table_names()

C:\Users\yhuan\AppData\Local\Temp/ipykernel_17548/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['people', 'fauna_vertabrate', 'us_gas_price', 'car_mpg']

In [16]:
#write us gas price df to postgres
avg_gas_price.to_sql(name='us_gas_price', con=engine, if_exists='replace', index=True)

In [17]:
#write car mpg df to postgres
avg_mpg.to_sql(name='car_mpg', con=engine, if_exists='replace', index=True)

In [18]:
pd.read_sql_query('select * from us_gas_price', con=engine).head()

,Year,All Grades All Formulations,All Grades Conventional,All Grades Reformulated,Regular All Formulations,Regular Conventional,Regular Reformulated,Midgrade All Formulations,Midgrade Conventional,Midgrade Reformulated,Premium All Formulations,Premium Conventional,Premium Reformulated,No 2 Diesel
0,2013,3.575365,3.510519,3.707442,3.504865,3.442865,3.634981,3.662577,3.584038,3.814577,3.822462,3.765231,3.928692,3.921731
1,2014,3.436615,3.376423,3.559288,3.357673,3.298788,3.481365,3.539423,3.469154,3.675365,3.710192,3.663923,3.796250,3.824635
2,2015,2.519923,2.422692,2.718000,2.429212,2.333750,2.629288,2.644827,2.530865,2.865442,2.830192,2.750385,2.978538,2.706635
3,2016,2.249962,2.177173,2.398000,2.143250,2.070462,2.296096,2.394615,2.312904,2.552404,2.615750,2.567269,2.705750,2.303942
4,2017,2.527885,2.447769,2.691231,2.414615,2.333173,2.585500,2.681615,2.595096,2.848885,2.916500,2.864577,3.013019,2.650038


In [19]:
pd.read_sql_query('select * from car_mpg', con=engine).head()

,Year,City FE (Guide) - Conventional Fuel,Hwy FE (Guide) - Conventional Fuel,Comb FE (Guide) - Conventional Fuel,City Unrd Adj FE - Conventional Fuel,Hwy Unrd Adj FE - Conventional Fuel,Comb Unrd Adj FE - Conventional Fuel
0,2013,19.574871,26.977625,22.246988,19.536401,26.912754,22.239994
1,2014,19.855351,27.418060,22.597826,19.839688,27.408190,22.599637
2,2015,20.153477,27.924061,22.961631,20.132416,27.876310,22.954063
3,2016,20.430338,28.164880,23.258862,20.424168,28.132987,23.241010
4,2017,20.569936,27.795016,23.245981,20.569113,27.772557,23.231466
